In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
images_path=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        images_path.append(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.__version__

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'2.12.0'

In [3]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import 







In [ ]:
# Create an ImageDataGenerator object for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to the range [0, 1]
    shear_range=0.2,             # Apply random shear transformations
    zoom_range=0.2,              # Apply random zoom transformations
    horizontal_flip=True        # Flip images horizontally
)

# Generate a flow of training data from the directory
training_set = train_datagen.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/train',   # Directory containing the training images
    target_size=(64, 64),                              # Resize images to (64, 64) pixels
    batch_size=10,                                     # Number of images per batch
    class_mode='binary'                                # Type of classification task (binary in this case)
)

In [ ]:
# Create an ImageDataGenerator object for test data preprocessing
test_datagen = ImageDataGenerator(
    rescale=1./255  # Rescale pixel values to the range [0, 1]
)

# Generate a flow of test data from the directory
test_set = test_datagen.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/test',  # Directory containing the test images
    target_size=(64, 64),                             # Resize images to (64, 64) pixels
    batch_size=10,                                    # Number of images per batch
    class_mode='binary'                               # Type of classification task (binary in this case)
)

In [4]:
import os
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

In [5]:
# Define image dimensions
img_width, img_height = 128, 128

# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = load_img(os.path.join(folder, filename), target_size=(img_width, img_height))
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

In [6]:
# Load benign and malignant train images
benign_folder_train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign'
malignant_folder_train ='/kaggle/input/skin-cancer-malignant-vs-benign/train/malignant'

benign_images_train = load_images_from_folder(benign_folder_train)
malignant_images_train = load_images_from_folder(malignant_folder_train)

In [9]:
# Load benign and malignant train images
benign_folder_test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/benign'
malignant_folder_test ='/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant'

benign_images_test = load_images_from_folder(benign_folder_test)
malignant_images_test = load_images_from_folder(malignant_folder_test)

In [10]:
# Create labels for the images
benign_labels_train = np.zeros(len(benign_images_train))
malignant_labels_train = np.ones(len(malignant_images_train))

benign_labels_test = np.zeros(len(benign_images_test))
malignant_labels_test = np.ones(len(malignant_images_test))


# Concatenate images and labels
#train_images, test_images, train_labels, test_labels
train_images = np.concatenate((benign_images_train, malignant_images_train), axis=0)
train_labels = np.concatenate((benign_labels_train, malignant_labels_train), axis=0)


test_images = np.concatenate((benign_images_test, malignant_images_test), axis=0)
test_labels = np.concatenate((benign_labels_test, malignant_labels_test), axis=0)

Apply autoencoder to reconstruct images, then add dense layers after the last Idecoder layer for classification.

In [11]:
# Autoencoder model
input_img = Input(shape=(img_width, img_height, 3))
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(encoded)
decoded = UpSampling2D((2, 2))(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Classification model
classification_input = autoencoder.input
classification_output = autoencoder.layers[-2].output
classification_output = Flatten()(classification_output)
classification_output = Dense(128, activation='relu')(classification_output)
classification_output = Dense(1, activation='sigmoid')(classification_output)

classification_model = Model(classification_input, classification_output)
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.summary()

# Train the autoencoder
autoencoder.fit(train_images, train_images, epochs=10, batch_size=32, validation_data=(test_images, test_images))

# Train the classification model
classification_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 3)         435       
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 128)               1572992   
                                                           

In [24]:
import os
import cv2
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array

# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize image to desired dimensions
            img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
            img = img.astype('float32') / 255.0
            images.append(img)
    return np.array(images)


# Load benign and malignant train images
benign_folder_train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign'
malignant_folder_train ='/kaggle/input/skin-cancer-malignant-vs-benign/train/malignant'

benign_images_train = load_images_from_folder(benign_folder_train)
malignant_images_train = load_images_from_folder(malignant_folder_train)


# Load benign and malignant train images
benign_folder_test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/benign'
malignant_folder_test ='/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant'

benign_images_test = load_images_from_folder(benign_folder_test)
malignant_images_test = load_images_from_folder(malignant_folder_test)





Apply autoencoder to reconstruct images, then add dense layers that takes the output of the encoder (bottleneck as input) for classification.

In [27]:
#Apply autoencoder to reconstruct images, then add dense layers that takes the output of the encoder (bottleneck ) for classification
# Concatenate images and create labels
x = np.concatenate((benign_images_train, malignant_images_train), axis=0)
y = np.concatenate((np.zeros(len(benign_images_train)), np.ones(len(malignant_images_train))), axis=0)
y = np.expand_dims(y, axis=-1)  # Reshape labels to match the classification output shape

# Autoencoder model
input_img = Input(shape=(128, 128, 3))
encoded = Dense(128, activation='relu')(input_img)
decoded = Dense(3, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

# Encoder model (bottleneck)
encoder = Model(input_img, encoded)

# Dense layers for classification
flatten = Flatten()(encoded)
classification_output = Dense(1, activation='sigmoid')(flatten)

classification_model = Model(input_img, classification_output)

# Compile and train the classification model
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.fit(x, y, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
66/66 [==============================] - 4s 44ms/step - loss: 0.6668 - accuracy: 0.6411 - val_loss: 0.9307 - val_accuracy: 0.0000e+00
Epoch 2/10
66/66 [==============================] - 2s 38ms/step - loss: 0.6302 - accuracy: 0.6828 - val_loss: 1.0212 - val_accuracy: 0.0000e+00
Epoch 3/10
66/66 [==============================] - 2s 38ms/step - loss: 0.6278 - accuracy: 0.6828 - val_loss: 1.2002 - val_accuracy: 0.0000e+00
Epoch 4/10
66/66 [==============================] - 3s 38ms/step - loss: 0.6242 - accuracy: 0.6828 - val_loss: 1.0419 - val_accuracy: 0.0000e+00
Epoch 5/10
66/66 [==============================] - 3s 38ms/step - loss: 0.6250 - accuracy: 0.6828 - val_loss: 0.9573 - val_accuracy: 0.0000e+00
Epoch 6/10
66/66 [==============================] - 2s 38ms/step - loss: 0.6241 - accuracy: 0.6828 - val_loss: 1.1015 - val_accuracy: 0.0000e+00
Epoch 7/10
66/66 [==============================] - 3s 40ms/step - loss: 0.6238 - accuracy: 0.6828 - val_loss: 1.2081 - val_accura

In [32]:
import os
import cv2
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Reshape
from keras.models import Model

# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize image to desired dimensions
            img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
    return np.array(images)

# Load benign and malignant train images
benign_folder_train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign'
malignant_folder_train ='/kaggle/input/skin-cancer-malignant-vs-benign/train/malignant'

benign_images_train = load_images_from_folder(benign_folder_train)
malignant_images_train = load_images_from_folder(malignant_folder_train)


# Load benign and malignant train images
benign_folder_test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/benign'
malignant_folder_test ='/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant'

benign_images_test = load_images_from_folder(benign_folder_test)
malignant_images_test = load_images_from_folder(malignant_folder_test)




After applying autoencoder, reshape the output vector of the last decoder layer and then add CNN layers and finally add Dense layers for classification

In [35]:
# Autoencoder model
input_img = Input(shape=(img_width, img_height, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(x)

# Decoder layers
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Reshape the output vector of the last decoder layer
flatten_layer = Flatten()(encoded)
reshaped_output = Reshape((8, 8, 8))(flatten_layer)

# CNN layers
cnn = Conv2D(16, (3, 3), activation='relu', padding='same')(reshaped_output)
cnn = MaxPooling2D((2, 2), padding='same')(cnn)
cnn = Conv2D(32, (3, 3), activation='relu', padding='same')(cnn)
cnn = MaxPooling2D((2, 2), padding='same')(cnn)

# Flatten and dense layers for classification
flatten = Flatten()(cnn)
dense = Dense(128, activation='relu')(flatten)
output = Dense(1, activation='sigmoid')(dense)

classification_model = Model(input_img, output)
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.summary()

# Train the classification model
classification_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

ValueError: Exception encountered when calling layer "reshape_7" (type Reshape).

total size of new array must be unchanged, input_shape = [32768], output_shape = [8, 8, 8]

Call arguments received by layer "reshape_7" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 32768), dtype=float32)

After applying autoencoder, reshape the output vector of the last decoder layer and then add Resnetse (transfer learning), CNN layers and finally add Dense layers for classification.

In [36]:
# Autoencoder model
input_img = Input(shape=(128, 128, 3))
encoded = Dense(128, activation='relu')(input_img)
decoded = Dense(16 * 16 * 3, activation='relu')(encoded)
reshaped_output = Reshape((16, 16, 3))(decoded)  # Reshape the output to (16, 16, 3)

# Transfer learning with ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=reshaped_output)
resnet_output = resnet_model.output

# CNN layers
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(resnet_output)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
flatten = Flatten()(pool2)

# Dense layers for classification
classification_output = Dense(1, activation='sigmoid')(flatten)

classification_model = Model(input_img, classification_output)

# Compile and train the classification model
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.fit(x, y, batch_size=32, epochs=10, validation_split=0.2)

ValueError: Exception encountered when calling layer "reshape_8" (type Reshape).

total size of new array must be unchanged, input_shape = [128, 128, 768], output_shape = [16, 16, 3]

Call arguments received by layer "reshape_8" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 128, 128, 768), dtype=float32)

Apply only CNN

In [2]:
# Apply only CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Check TensorFlow version
tf.__version__

# Data augmentation and preprocessing for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load and preprocess the training set
training_set = train_datagen.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/train',
    target_size=(64, 64),
    batch_size=10,
    class_mode='binary'
)

# Data preprocessing for the test set
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Load and preprocess the test set
test_set = test_datagen.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/test',
    target_size=(64, 64),
    batch_size=10,
    class_mode='binary'
)


Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [42]:
import tensorflow as tf

# Create a Sequential model
cnn = tf.keras.models.Sequential()

# Add a convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Add a max pooling layer with a pool size of 2x2 and stride of 2
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Add another convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

# Add another max pooling layer with a pool size of 2x2 and stride of 2
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flatten the output from the previous layer
cnn.add(tf.keras.layers.Flatten())

# Add a dense layer with 128 units and ReLU activation
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Add an output layer with 1 unit (for binary classification) and sigmoid activation
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model with Adam optimizer, binary cross-entropy loss, and accuracy metric
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the training set and validate on the test set for 50 epochs
cnn.fit(x=training_set, validation_data=test_set, epochs=50)


Epoch 1/50
264/264 [==============================] - 16s 53ms/step - loss: 0.6332 - accuracy: 0.6231 - val_loss: 0.5029 - val_accuracy: 0.7682
Epoch 2/50
264/264 [==============================] - 12s 47ms/step - loss: 0.4924 - accuracy: 0.7637 - val_loss: 0.4126 - val_accuracy: 0.7955
Epoch 3/50
264/264 [==============================] - 13s 48ms/step - loss: 0.4434 - accuracy: 0.7744 - val_loss: 0.4074 - val_accuracy: 0.8015
Epoch 4/50
264/264 [==============================] - 13s 50ms/step - loss: 0.4046 - accuracy: 0.7983 - val_loss: 0.4129 - val_accuracy: 0.7939
Epoch 5/50
264/264 [==============================] - 13s 48ms/step - loss: 0.3974 - accuracy: 0.8074 - val_loss: 0.3810 - val_accuracy: 0.8015
Epoch 6/50
264/264 [==============================] - 14s 52ms/step - loss: 0.4083 - accuracy: 0.7956 - val_loss: 0.3839 - val_accuracy: 0.8030
Epoch 7/50
264/264 [==============================] - 13s 49ms/step - loss: 0.4031 - accuracy: 0.8009 - val_loss: 0.3711 - val_accuracy:

we load the ResNet50 model with pre-trained weights from the 'imagenet' dataset. We set include_top=False to exclude the top classification layers of ResNet50. The ResNet50 model is then added as the first layer of the CNN model.

The CNN layers are defined as before, with a Flatten layer followed by two dense layers for classification.

Finally, the model is compiled and trained using the training_set and test_set data.

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

# Load ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
resnet_model.trainable = False

# Create CNN model
cnn = tf.keras.models.Sequential()
cnn.add(resnet_model)
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
cnn.fit(x=training_set, validation_data=test_set, epochs=50)

Epoch 1/50
264/264 [==============================] - 44s 125ms/step - loss: 0.6980 - accuracy: 0.5544 - val_loss: 0.6652 - val_accuracy: 0.7182
Epoch 2/50
264/264 [==============================] - 12s 44ms/step - loss: 0.6586 - accuracy: 0.6030 - val_loss: 0.6453 - val_accuracy: 0.6258
Epoch 3/50
264/264 [==============================] - 12s 47ms/step - loss: 0.6373 - accuracy: 0.6435 - val_loss: 0.5941 - val_accuracy: 0.6742
Epoch 4/50
264/264 [==============================] - 12s 44ms/step - loss: 0.6039 - accuracy: 0.6811 - val_loss: 0.5610 - val_accuracy: 0.7530
Epoch 5/50
264/264 [==============================] - 12s 46ms/step - loss: 0.5815 - accuracy: 0.6978 - val_loss: 0.5510 - val_accuracy: 0.6773
Epoch 6/50
264/264 [==============================] - 12s 45ms/step - loss: 0.5712 - accuracy: 0.7065 - val_loss: 0.5258 - val_accuracy: 0.7379
Epoch 7/50
264/264 [==============================] - 12s 47ms/step - loss: 0.5652 - accuracy: 0.7072 - val_loss: 0.5225 - val_accuracy